In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

#Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# dx = 1 km; Np = 1M; Nt = 5 min
data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 100M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_100M.nc') #***
# res='1km'; t_res='1min'; Np_str='100e6'


# dx = 250 m
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_250m.nc') #***

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(PlottingFunctions, inspect.isfunction)]
# functions

In [ ]:
############################################################
#PLOTTING
plotting=False #KEEP FALSE IF JOB ARRAY IS RUNNING
plotting=True

In [ ]:
if plotting==True:
    #constants
    Cp=1004 #Jkg-1K-1
    Cv=717 #Jkg-1K-1
    Rd=Cp-Cv #Jkg-1K-1
    eps=0.608
    
    Lx=(data['xf'][-1].item()-data['xf'][0].item())*1000 #x length (m)
    Ly=(data['yf'][-1].item()-data['yf'][0].item())*1000 #y length (m)
    Np=len(parcel['xh']) #number of lagrangian parcles
    dt=(data['time'][1]-data['time'][0]).item()/1e9 #sec
    dx=(data['xf'][1].item()-data['xf'][0].item())*1e3 #meters
    dy=(data['yf'][1].item()-data['yf'][0].item())*1e3 #meters
    xs=data['xf'].values*1000
    ys=data['yf'].values*1000
    zs=data['zf'].values*1000
    
    def zf(z):
        k=z #z is the # level of z
        out=data['zf'].values[k]*1000
        
        return out
    # def rho(x,y,z,t):
    #     p=data['prs'].isel(xh=x,yh=y,zh=z,time=t).item()
    #     p0=101325 #Pa
    #     theta=data['th'].isel(xh=x,yh=y,zh=z,time=t).item()
    #     T=theta*(p/p0)**(Rd/Cp)
    #     qv=data['qv'].isel(xh=x,yh=y,zh=z,time=t).item()
    #     # Tv=T*(1+eps*qv)
    #     Tv=T*(eps+qv)/(eps*(1+qv))
    #     rho = p/(Rd*Tv)
    #     out=rho
    #     return out
    
    def rho(x,y,z,rho_data_t):
        out=rho_data_t[z,y,x]
        return out
    def m(t):
        rho_data_t=data['rho'].isel(time=t).data
        
        m=0
        #triple sum
        for k in range(len(data['zh'])):
            dz=(zf(k+1)-zf(k))
            for j in range(len(data['yh'])):
                for i in range(len(data['xh'])):
                    rho_out=rho(i,j,k,rho_data_t)
                    m+=rho_out*dz
                    
        #triple sum
        out=m*dx*dy/Np
        return out


In [ ]:
#SOME CALCULATIONS (TESTING)
# # (Lx*Ly*(10**4))/1e6 #1M parcels ==> 1 billion kg/parcel
# # (Lx*Ly*(10**4))/50e6 #50M parcels ==> 20 million kg/parcel
# # (Lx*Ly*(10**4))/100e6 #100M parcels ==> 10 million kg/parcel

# # 1e5 kg | 9.1125 e4 m^3
# # x   kg | 1000*1000*62 = 6.2e7 m^3
# (1000*1000*62)*(1e5/(9.1125e4))# ==> 68038408 ==> should have 68M kg in the bottom most layer 

# 68038408/19729158# (expected/calculated mass) ==> should have 3.5 parcels in each grid box on the bottom most layer
# #we have 369e3 parcels on bottom layer ==> 369e3/(Nx*Ny) = 369e3/102400 = 3.6 parcels per layer!

In [ ]:
if plotting==True:
    #Calculate Mass Constant
    # calculate='single_time'
    # calculate=True
    calculate=False
    
    if calculate==True:
        Nt=len(data['time'])
        m_arr=np.zeros((Nt))
        for t in np.arange(Nt):
            if np.mod(t,25)==0: print(t)
            m_arr[t]=m(t)
        dir3=dir+f'Project_Algorithms/Entrainment/'
        np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
    elif calculate=='single_time':
        Nt=len(data['time'])
        m_arr=np.zeros((Nt))
    
        t=0 #len(data['time'])//2 #Pick some middle time
        m_300=m(t)
        for t in np.arange(Nt):
            m_arr[t]=m_300 #UNCOMMENT FOR FULL CALCULATION
        dir3=dir+f'Project_Algorithms/Entrainment/'
        np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
    else:
        dir3=dir+f'Project_Algorithms/Entrainment/'
        m_arr = np.load(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy')
    
    # # TESTING
    # lst=[]
    # for t in np.arange(133):
    #     lst.append(m_arr[t])
    
    # plt.plot(lst)
    # (np.max(lst)-np.min(lst))*100/np.mean(lst)

In [ ]:
if plotting==True:
    PROCESSING=False
    PROCESSING=True
    
    dir3=dir+'Project_Algorithms/Entrainment/'
    if PROCESSING==False:
        open_file=dir3+f'2D_entrainmentdetrainment_combined2_profiles_{res}_{t_res}_{Np_str}.h5'
    if PROCESSING==True:
        open_file=dir3+f'2D_entrainmentdetrainment_combined2_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
    with h5py.File(open_file, "r") as h5f:
        profile_array_e_g = h5f["profile_array_e_g"][:]
        profile_array_e_c = h5f["profile_array_e_c"][:]
        profile_array_d_g = h5f["profile_array_d_g"][:]
        profile_array_d_c = h5f["profile_array_d_c"][:]

In [ ]:
if plotting==True:
    type='general'
    type='cloudy'
    
    def apply_constant(profile_array,apply):
        if apply==True:
            Nt=profile_array.shape[0]
            Nz=profile_array.shape[1]
        
            profile_array/=(Lx*Ly*dt)
            for t in np.arange(Nt):
                profile_array[t]*=m_arr[t]
            for z in np.arange(Nz):
                dz=zf(z+1)-zf(z)
                profile_array[:,z]/=dz
        return profile_array
    #APPLY CONSTANTS TO ENTRAINMENT VALUE
    ##################################################
    profile_array_e_g=apply_constant(profile_array_e_g,apply=True)
    profile_array_e_c=apply_constant(profile_array_e_c,apply=True)
    profile_array_d_g=apply_constant(profile_array_d_g,apply=True)
    profile_array_d_c=apply_constant(profile_array_d_c,apply=True)
    ##################################################
    
    if type=='general':
        profile_array_e=profile_array_e_g
        profile_array_d=profile_array_d_g
        profile_array_net=profile_array_e-profile_array_d
    if type=='cloudy':
        profile_array_e=profile_array_e_c
        profile_array_d=profile_array_d_c
        profile_array_net=profile_array_e-profile_array_d

In [ ]:
if plotting==True:
    #Plotting
    ############################################################
    import matplotlib.pyplot as plt
    from matplotlib.gridspec import GridSpec
    import numpy as np
    
    fig = plt.figure(figsize=(10, 8))
    gs = GridSpec(2, 2, figure=fig)
    
    ######
    cmap1 = plt.cm.viridis
    cmap2 = plt.cm.seismic 
    n_levels=29
    ######
    
    ######
    vmax_shared = np.max([np.max(profile_array_e), np.max(profile_array_d)])
    norm_shared = mcolors.Normalize(vmin=0, vmax=vmax_shared)
    ######
    
    # First subplot: Entrainment
    ########################################
    ax1 = fig.add_subplot(gs[0, 0])
    # contour1 = ax1.contourf(profile_array_e.T, cmap=cmap1)
    contour1 = ax1.contourf(profile_array_e.T, cmap=cmap1, norm=norm_shared, levels=n_levels)
    cbar1=fig.colorbar(contour1, ax=ax1)
    Nz = len(data['zh'])
    ax1.set_yticks(np.arange(Nz))
    new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
    ax1.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
    ax1.set_ylabel('z (km)');ax1.set_xlabel('t (timesteps)')
    ax1.set_title('Entrainment using Lagrangian Binary Array',fontsize=8)
    
    # Second subplot: Detrainment
    ########################################
    ax2 = fig.add_subplot(gs[0, 1])
    # contour2 = ax2.contourf(profile_array_d.T, cmap=cmap1)
    contour2 = ax2.contourf(profile_array_d.T, cmap=cmap1, norm=norm_shared, levels=n_levels)
    cbar2 = fig.colorbar(contour2, ax=ax2)
    ax2.set_yticks(np.arange(Nz))
    new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
    ax2.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
    ax2.set_ylabel('z (km)');ax2.set_xlabel('t (timesteps)')
    ax2.set_title('Detrainment')
    
    # Third subplot: Net Entrainment
    ########################################
    
    
    # #OLD METHOD, DOESNT BALANCE COLOR LEVELS
    # # Normalize with a balanced vmin and vmax
    # levels=49; vmin=np.min(profile_array_net);vmax=np.max(profile_array_net)
    # # vmin=-np.max(abs(profile_array_net)); vmax=+np.max(abs(profile_array_net))
    # norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
    
    # Normalize with a balanced vmin and vmax
    vmin=-np.max(abs(profile_array_net))/2; vmax=+np.max(abs(profile_array_net))

    percentile_vminmax=False
    if percentile_vminmax==True:
        ####
        vmin = np.percentile(profile_array_net[profile_array_net<0], 1)
        vmax = np.percentile(profile_array_net[profile_array_net>0], 99)
        ####

    
        
    levels = np.linspace(vmin, vmax, n_levels)
    norm = mcolors.BoundaryNorm(boundaries=levels, ncolors=256)
    cmap = plt.get_cmap('RdBu_r', n_levels)
    
    ax3 = fig.add_subplot(gs[1, 0])
    contour3 = ax3.contourf((profile_array_net).T, cmap=cmap2, norm=norm, levels=levels)
    # contour3 = ax3.contourf((profile_array_net).T, cmap=cmap2, levels=30,vmin=-np.max(abs(profile_array_net)), vmax=+np.max(abs(profile_array_net)))
    # cmap2 = plt.get_cmap('RdBu', 29);contour3 = ax3.pcolor(profile_array_net.T, cmap=cmap2, norm=norm, shading='auto')
    cbar3 = fig.colorbar(contour3, ax=ax3, norm=norm)
    
    #FIXING TICKS
    ax3.set_yticks(np.arange(Nz))
    new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
    ax3.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
    ax3.set_ylabel('z (km)');ax3.set_xlabel('t (timesteps)')
    ax3.set_title('Entrainment - Detrainment')
    
    #FIXING SCIENTIFIC NOTATION
    
    def apply_scientific_notation_colorbar(cbars):
        from matplotlib.ticker import ScalarFormatter
        formatter = ScalarFormatter(useMathText=True)
        formatter.set_powerlimits((-2, 2))  # Adjust the range for scientific notation
        for cbar in cbars:  # These must be Colorbar instances
            cbar.formatter = formatter
            cbar.update_ticks()
    apply_scientific_notation_colorbar([cbar1,cbar2,cbar3])
    
    # Display the plot
    plt.tight_layout()
    
    #TESTING
    print(f"Max of profile_array_e: {np.max(profile_array_e)}")
    print(f"Max of profile_array_d: {np.max(profile_array_d)}")

In [ ]:
if plotting==True:
    e=np.mean(profile_array_e,axis=(0))
    d=np.mean(profile_array_d,axis=(0))
    net=np.mean(profile_array_e-profile_array_d,axis=(0))
    
    plt.plot(e,data['zh'],color='blue',label='entrainment')
    plt.plot(d,data['zh'],color='red',label='detrainment')
    plt.plot(net,data['zh'],linestyle='dashed',color='black',label='entrainment - detrainment')
    plt.axvline(0,color='black')
    
    plt.legend(); plt.title('2D Entrainment and Detrainment Using Lagrangian Binary Array')
    
    from matplotlib.ticker import ScalarFormatter
    formatter = ScalarFormatter(useMathText=True)
    formatter.set_scientific(True)
    formatter.set_powerlimits((-1, 1))
    plt.gca().xaxis.set_major_formatter(formatter)